In [20]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [5]:
sample_info_df = pd.read_csv('data/sample_info.csv')
sample_info_df.head()

,DepMap_ID,cell_line_name,stripped_cell_line_name,CCLE_Name,alias,COSMICID,sex,source,RRID,WTSI_Master_Cell_ID,...,lineage_sub_subtype,lineage_molecular_subtype,default_growth_pattern,model_manipulation,model_manipulation_details,patient_id,parent_depmap_id,Cellosaurus_NCIt_disease,Cellosaurus_NCIt_id,Cellosaurus_issues
0,ACH-000016,SLR 21,SLR21,SLR21_KIDNEY,NaN,NaN,NaN,Academic lab,CVCL_V607,NaN,...,NaN,NaN,NaN,NaN,NaN,PT-JnARLB,NaN,Clear cell renal cell carcinoma,C4033,NaN
1,ACH-000032,MHH-CALL-3,MHHCALL3,MHHCALL3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,NaN,Female,DSMZ,CVCL_0089,NaN,...,b_cell,NaN,NaN,NaN,NaN,PT-p2KOyI,NaN,Childhood B acute lymphoblastic leukemia,C9140,NaN
2,ACH-000033,NCI-H1819,NCIH1819,NCIH1819_LUNG,NaN,NaN,Female,Academic lab,CVCL_1497,NaN,...,NSCLC_adenocarcinoma,NaN,NaN,NaN,NaN,PT-9p1WQv,NaN,Lung adenocarcinoma,C3512,NaN
3,ACH-000043,Hs 895.T,HS895T,HS895T_FIBROBLAST,NaN,NaN,Female,ATCC,CVCL_0993,NaN,...,NaN,NaN,2D: adherent,NaN,NaN,PT-rTUVZQ,NaN,Melanoma,C3224,NaN
4,ACH-000049,HEK TE,HEKTE,HEKTE_KIDNEY,NaN,NaN,NaN,Academic lab,CVCL_WS59,NaN,...,NaN,NaN,NaN,immortalized,NaN,PT-qWYYgr,NaN,NaN,NaN,No information is available about this cell li...


In [6]:
sample_info_df.columns

Index(['DepMap_ID', 'cell_line_name', 'stripped_cell_line_name', 'CCLE_Name',
       'alias', 'COSMICID', 'sex', 'source', 'RRID', 'WTSI_Master_Cell_ID',
       'sample_collection_site', 'primary_or_metastasis', 'primary_disease',
       'Subtype', 'age', 'Sanger_Model_ID', 'depmap_public_comments',
       'lineage', 'lineage_subtype', 'lineage_sub_subtype',
       'lineage_molecular_subtype', 'default_growth_pattern',
       'model_manipulation', 'model_manipulation_details', 'patient_id',
       'parent_depmap_id', 'Cellosaurus_NCIt_disease', 'Cellosaurus_NCIt_id',
       'Cellosaurus_issues'],
      dtype='object')

In [13]:
crispr_gene_effect_df = pd.read_csv('data/CRISPR_gene_effect.csv')
crispr_gene_effect_df

,DepMap_ID,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZWILCH (55055),ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,ACH-000001,-0.134808,0.059764,-0.008665,-0.003572,-0.106211,-0.008257,0.018711,-0.291985,0.010921,...,-0.037619,-0.116524,-0.029331,0.105940,0.147605,-0.119822,0.063387,0.160857,0.058648,-0.316792
1,ACH-000004,0.081853,-0.056401,-0.106738,-0.014499,0.078209,-0.137562,0.168657,-0.198560,0.133372,...,-0.030901,-0.262220,0.136406,0.031327,0.093763,-0.079692,-0.173709,0.153632,0.175627,-0.040869
2,ACH-000005,-0.094196,-0.014598,0.100426,0.169103,0.032363,-0.148050,0.168931,-0.244777,-0.086871,...,0.039434,-0.336925,-0.095528,-0.035541,-0.035612,-0.040183,-0.165464,0.077343,0.019387,-0.085687
3,ACH-000007,-0.011544,-0.123189,0.080692,0.061046,-0.013454,-0.016922,-0.029474,-0.206516,-0.063998,...,-0.229303,-0.463191,0.061641,0.190301,0.119388,-0.036695,-0.182449,-0.146936,-0.189451,-0.281167
4,ACH-000009,-0.050782,-0.037466,0.068885,0.090375,0.012634,-0.079339,-0.017808,-0.183192,0.006227,...,-0.157219,-0.318765,0.015761,0.196949,-0.045874,-0.186805,-0.275629,-0.001227,-0.049140,-0.240582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,ACH-002834,0.093856,-0.141460,0.088443,0.088377,-0.175830,-0.133436,0.088166,-0.681595,-0.045118,...,0.002824,-0.588608,0.026231,-0.080376,-0.007511,-0.008117,0.017214,-0.249746,-0.153472,-0.426827
1082,ACH-002847,-0.041373,-0.074384,0.005141,0.023614,-0.060251,0.015849,0.103120,-0.365970,-0.096872,...,-0.365993,-1.057614,0.093102,0.068145,0.119134,-0.064060,-0.134209,-0.117938,-0.093320,-0.343878
1083,ACH-002874,-0.141663,0.087888,0.084764,0.175138,-0.097428,-0.050125,-0.065607,-0.121013,-0.117815,...,-0.465202,-0.492154,-0.000445,0.001417,0.085010,-0.052544,-0.072800,-0.138826,-0.024263,-0.228572
1084,ACH-002875,0.010012,-0.096187,-0.060130,-0.020200,-0.214935,-0.149155,-0.013285,-0.262853,-0.083522,...,-0.265429,-0.160332,-0.034464,-0.000718,0.076519,-0.097779,-0.081805,-0.025186,-0.170965,-0.277125


In [14]:
ccle_expression_df = pd.read_csv('data/CCLE_expression.csv')
ccle_expression_df.head()

,Unnamed: 0,TSPAN6 (7105),TNMD (64102),DPM1 (8813),SCYL3 (57147),C1orf112 (55732),FGR (2268),CFH (3075),FUCA2 (2519),GCLC (2729),...,H3C2 (8358),H3C3 (8352),AC098582.1 (8916),DUS4L-BCAP29 (115253422),C8orf44-SGK3 (100533105),ELOA3B (728929),NPBWR1 (2831),ELOA3D (100506888),ELOA3 (162699),CDR1 (1038)
0,ACH-001113,4.331992,0.000000,7.364397,2.792855,4.470537,0.028569,1.226509,3.042644,6.499686,...,2.689299,0.189034,0.201634,2.130931,0.555816,0.0,0.275007,0.0,0.0,0.000000
1,ACH-001289,4.566815,0.584963,7.106537,2.543496,3.504620,0.000000,0.189034,3.813525,4.221104,...,1.286881,1.049631,0.321928,1.464668,0.632268,0.0,0.014355,0.0,0.0,0.000000
2,ACH-001339,3.150560,0.000000,7.379032,2.333424,4.227279,0.056584,1.310340,6.687061,3.682573,...,0.594549,1.097611,0.831877,2.946731,0.475085,0.0,0.084064,0.0,0.0,0.042644
3,ACH-001538,5.085340,0.000000,7.154109,2.545968,3.084064,0.000000,5.868143,6.165309,4.489928,...,0.214125,0.632268,0.298658,1.641546,0.443607,0.0,0.028569,0.0,0.0,0.000000
4,ACH-000242,6.729145,0.000000,6.537607,2.456806,3.867896,0.799087,7.208381,5.569856,7.127014,...,1.117695,2.358959,0.084064,1.910733,0.000000,0.0,0.464668,0.0,0.0,0.000000


In [15]:
ccle_gene_cn_df = pd.read_csv('data/CCLE_gene_cn.csv')
ccle_gene_cn_df.head()

,Unnamed: 0,DDX11L1 (84771),WASH7P (653635),MIR6859-1 (102466751),MIR1302-2 (100302278),FAM138A (645520),OR4F5 (79501),WASH9P (102723897),MIR6859-2 (102465909),OR4F29 (729759),...,TTTY17B (474151),TTTY4B (474149),BPY2B (442867),DAZ3 (57054),DAZ4 (57135),BPY2C (442868),TTTY4C (474150),CSPG4P1Y (114758),CDY1 (9085),TTTY3 (114760)
0,ACH-000267,1.149159,1.149159,1.149159,1.149159,1.149159,1.149159,1.149159,1.149159,1.149159,...,5.421627e-01,5.421627e-01,5.421627e-01,5.421627e-01,5.421627e-01,5.421627e-01,5.421627e-01,5.421627e-01,5.421627e-01,5.421627e-01
1,ACH-001408,1.039446,1.039446,1.039446,1.039446,1.039446,1.039446,1.039446,1.039446,1.039446,...,5.107920e-01,5.107920e-01,5.107920e-01,5.107920e-01,5.107920e-01,5.107920e-01,5.107920e-01,5.107920e-01,5.107920e-01,5.107920e-01
2,ACH-000617,0.762353,0.762353,0.762353,0.762353,0.762353,0.762353,0.762353,0.762353,0.762353,...,1.032871e-08,1.032871e-08,1.032871e-08,1.032871e-08,1.032871e-08,1.032871e-08,1.032871e-08,1.032871e-08,1.032871e-08,1.032871e-08
3,ACH-002123,1.137564,1.137564,1.137564,1.137564,1.137564,1.137564,1.137564,1.137564,1.137564,...,4.717688e-01,4.717688e-01,4.717688e-01,4.717688e-01,4.717688e-01,4.717688e-01,4.717688e-01,4.717688e-01,4.717688e-01,4.717688e-01
4,ACH-000519,1.005822,1.005822,1.005822,1.005822,1.005822,1.005822,1.005822,1.005822,1.005822,...,8.638107e-10,8.638107e-10,8.638107e-10,8.638107e-10,8.638107e-10,8.638107e-10,8.638107e-10,8.638107e-10,8.638107e-10,8.638107e-10


In [10]:
gene_data_with_lineage = merged_df = pd.merge(crispr_gene_effect_df, sample_info_df[['DepMap_ID', 'lineage']], on='DepMap_ID', how='left')

In [16]:
gene_data_with_lineage

,DepMap_ID,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009),lineage
0,ACH-000001,-0.134808,0.059764,-0.008665,-0.003572,-0.106211,-0.008257,0.018711,-0.291985,0.010921,...,-0.116524,-0.029331,0.105940,0.147605,-0.119822,0.063387,0.160857,0.058648,-0.316792,ovary
1,ACH-000004,0.081853,-0.056401,-0.106738,-0.014499,0.078209,-0.137562,0.168657,-0.198560,0.133372,...,-0.262220,0.136406,0.031327,0.093763,-0.079692,-0.173709,0.153632,0.175627,-0.040869,blood
2,ACH-000005,-0.094196,-0.014598,0.100426,0.169103,0.032363,-0.148050,0.168931,-0.244777,-0.086871,...,-0.336925,-0.095528,-0.035541,-0.035612,-0.040183,-0.165464,0.077343,0.019387,-0.085687,blood
3,ACH-000007,-0.011544,-0.123189,0.080692,0.061046,-0.013454,-0.016922,-0.029474,-0.206516,-0.063998,...,-0.463191,0.061641,0.190301,0.119388,-0.036695,-0.182449,-0.146936,-0.189451,-0.281167,colorectal
4,ACH-000009,-0.050782,-0.037466,0.068885,0.090375,0.012634,-0.079339,-0.017808,-0.183192,0.006227,...,-0.318765,0.015761,0.196949,-0.045874,-0.186805,-0.275629,-0.001227,-0.049140,-0.240582,colorectal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,ACH-002834,0.093856,-0.141460,0.088443,0.088377,-0.175830,-0.133436,0.088166,-0.681595,-0.045118,...,-0.588608,0.026231,-0.080376,-0.007511,-0.008117,0.017214,-0.249746,-0.153472,-0.426827,bone
1082,ACH-002847,-0.041373,-0.074384,0.005141,0.023614,-0.060251,0.015849,0.103120,-0.365970,-0.096872,...,-1.057614,0.093102,0.068145,0.119134,-0.064060,-0.134209,-0.117938,-0.093320,-0.343878,unknown
1083,ACH-002874,-0.141663,0.087888,0.084764,0.175138,-0.097428,-0.050125,-0.065607,-0.121013,-0.117815,...,-0.492154,-0.000445,0.001417,0.085010,-0.052544,-0.072800,-0.138826,-0.024263,-0.228572,skin
1084,ACH-002875,0.010012,-0.096187,-0.060130,-0.020200,-0.214935,-0.149155,-0.013285,-0.262853,-0.083522,...,-0.160332,-0.034464,-0.000718,0.076519,-0.097779,-0.081805,-0.025186,-0.170965,-0.277125,skin


In [21]:
## Train test split


# Features
X = gene_data_with_lineage.drop(columns=['lineage'])

# Target variable
y = gene_data_with_lineage['lineage']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Principal Component Analysis (PCA)

## Random Forest

## Neural Network

## Support Vector Machine (SVM)

## Logistic Regression